In [0]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATASETS
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S R
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

DATASET_MAPPING = 'bda-2023-facial-expressions:http%3A%2F%2Flocalhost%2Flocalgs%2Fv1%2Fdownload%2Fkaggle-competitions-data%2Fkaggle-v2%2F60511%2F6530139%2Fbundle%2Farchive.zip'
KAGGLE_INPUT_PATH = '/home/kaggle/input'
KAGGLE_INPUT_SYMLINK = '/kaggle'

system(paste0('sudo mkdir -p -- ', KAGGLE_INPUT_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_INPUT_PATH), intern=TRUE)
system(paste0('sudo ln -sfn ', KAGGLE_INPUT_PATH, ' ../'), intern=TRUE)
system(paste0('sudo mkdir -p -- ', KAGGLE_INPUT_SYMLINK))
system(paste0('sudo ln -sfn ', KAGGLE_INPUT_PATH, ' ', KAGGLE_INPUT_SYMLINK))

datasets_mappings = strsplit(DATASET_MAPPING, ',')[[1]]
for (dataset_mapping in datasets_mappings) {
    path_and_url = strsplit(dataset_mapping, ':')
    directory = path_and_url[[1]][1]
    download_url = URLdecode(path_and_url[[1]][2])
    destination_path = file.path(KAGGLE_INPUT_PATH, directory)
    temp = tempfile(fileext = '.zip')
    download.file(download_url, temp)
    unzip(temp, overwrite = TRUE, exdir = destination_path)
    unlink(temp)
    print(paste0('Downloaded and unzipped: ', directory))
}


# <center> **FACIAL EMOTION RECOGNITION**
    
# 1. Introduction

In this competition, we will build a classifier algorithm that categorizes facial expressions of humans in gray-scale portraits into one of four emotion cagetories: anger, happiness, disgust, sadness 
    
    
Before we get started, we will answer the following questions:
    
**Where do the data come from, and to which population will results generalize?**
    
> The dataset is called CKPlus. It is a  widely used facial expression database used for research in the field of facial expression analysis and computer vision. (S. Li, W. Deng, Deep facial expression recognition: a survey. IEEE Trans. Affect. Comput. 13, 1195–1215 (2020)). It consists of both posed and spontaneous expressions from a total of 123 subjects with diverse ethnic backgrounds aged between 18-50 years. The pictures were taken under controlled lighting conditions with subjects seated in front of a camera. Researchers coded expressed emotion , the intensity of the emotion, and the location of facial landmarks. They included seven different emotions: Anger, Contempt, Disgust, Fear, Happiness, Sadness, Surprise. 
>
> For our project, we only used a subset of the CL+ database. We only distinguished between four different emotions and did not include its intensity. Additionaly, some of the images are repeated, but then shifted, rotated, or both to improve the algorithm. 
>    
> While the datset did include subjects of different ethnicities and age, the generalizability of our results will still be limited. As metioned, the pictures were taken under controlled conditions with consistent and good lighting. This means that our algorithm will probably have difficulties recognizing facial expressions in more "normal", day-to-day pictures. In addition to that, the datapoints are not independent of each other as multiple pictures were taken of each participant. Independence, however, is an important assumption of many algorithms and statistical tests.     

**What are candidate machine learning methods?**

> There is a wide variety of possible algorithms to choose from. These algorithms differ in their levels of flexibility and bias. Possible algorithms sorted from least to most flexible are: 
> Linear Discrimminant Analysis, multinomial logistic regression, Ridge Regression, Lasso Regression, , K-nearest Neighbors, Random Forestn, Support Vector 
> 
> The more flexible a model is, the more it is driven by the training data and will probably achieve a good model fit. The "danger" of highly flexible models is that they *overfit* the training data which would mean that they do not predict future observations well, which is our goal. This is why we chose less flexible (aka more biased) models that make stronger assumptions about the underlying distribution. This might result it a worse model fit, but will likely predict future cases better. 
> This is why we decided on fitting ...

**What is the Bayes' error bound?**

> The Bayes' error bound represents a fundamental theoretical limit on the classification error rate (aka the minimym accuracy) that any algorithm should achieve. Previous research has determined the accuracy of humans when as 67% for disgust, 62% for anger, 80% for happy, 70% for sad. Human accuracy has also been tested for the CK+ database specifically which lead to the confusion matrix shown below and an overall accuracy of 74.4%. <br>
>
>|      | Anger | Disgust | Happy | Sadness |
>|------|-------|---------|-------|---------|
>| Anger   | 23   | 9       | 1     | 0       |
>| Disgust | 6    | 35      | 4     | 2       |
>| Happy   | 4    | 3       | 66    | 0       |
>| Sad     | 9    | 7       | 1     | 10      |
>
> For Machine Learning models the Bayes Error Bound is discussed in: https://www.hindawi.com/journals/cin/2022/8032673/ The reported accuracies for emotion recognition through facial expression range in the experiment have a training accuracy of 100% and a test accuracy of up to 90% for SVM and RF and up to 95% for KNN.
    
    
# 2. Data
    
    
## 2.1 Preparation
    
First, we loaded all necessary libraries, set the working directory and imported all image files. 
    

In [ ]:
## loading libraries

library(tidyverse) 
library(png) 
library(caret)
suppressMessages(library(tidytext))
suppressMessages(library(caret))
suppressMessages(library(glmnet))
suppressMessages(library(ggplot2))
suppressMessages(library(gridExtra))

## Reading in files

list.files(path = "../input/")


# Show availabe directories
dirs = dir("../input", pattern="[^g]$", recursive=TRUE, include.dirs = TRUE, full.names = TRUE)
# dirs

# Get all image files: file names ending ".png" 
anger   = dir(grep("anger",   dirs, value = TRUE), pattern = "png$", full.names = TRUE)
disgust = dir(grep("disgust", dirs, value = TRUE), pattern = "png$", full.names = TRUE)
happy   = dir(grep("happy",   dirs, value = TRUE), pattern = "png$", full.names = TRUE)
sad     = dir(grep("sad",     dirs, value = TRUE), pattern = "png$", full.names = TRUE)
test_im = dir(grep("test",    dirs, value = TRUE), pattern = "png$", full.names = TRUE)

## 2.2 Loading the data

Next, we loaded the data and created a function to visualize some of the pictures to make sure that everything worked. 

Each imagine is stored in a vector of pixel intensities. Another vector contains the emotion labels for each of the images. In order to save on RAM, the images for this competition are...

* ... gray scale, so we need only one color channel
* ... are only 48 by 48 pixels


In [ ]:
# Combine all filenames into a single vector
train_image_files = c(anger, happy, sad, disgust)

# Read in the images as pixel values (discarding color channels)
X = sapply(train_image_files, function(nm) c(readPNG(nm)[,,1])) %>% t() 
y = c(rep("anger", length(anger)), rep("happy", length(happy)), rep("sad", length(sad)), rep("disgust", length(disgust)))

X_test = sapply(test_im, function(nm) c(readPNG(nm)[,,1])) %>% t() 


# Change row and column names of X to something more managable (caret::train requires column names)
rownames(X)      = gsub(".+train/", "", rownames(X))
rownames(X_test) = gsub(".+test/",  "", rownames(X_test))

colnames(X) = colnames(X_test) = paste("p",1:ncol(X), sep="")

# Check result (are X, X_test, and y what we expect)
# X[1:6,20:23] %>% print
# table(y)
# X_test[1:6,20:23] %>% print
                
                
# Visualization utility function
as_image = function(x, nr=sqrt(length(x))) {opar=par(mar=rep(0,4)); on.exit(par(opar)); image(t(matrix(x,nr))[,nr:1], col = gray(0:255/255),axes=F)}

## 2.3 Creating a Test Subset

Lastly, we split the data into a test and training set to evaluate our model's performance using a confusion matrix. 

In [ ]:
set.seed(123)

# Create random partition of the data
train_val = caret::createDataPartition(y, p = 0.8, list = FALSE)

# Define training & validation set
train_X = X[train_val, ]
#train_X = X[sample(train_val, 500),] # REMOVE THIS FOR CREATING AN ACTUAL SUBMISSION F
train_y = y[train_val]

test_X = X[-train_val, ]
test_y = y[-train_val]

X <- rbind(train_X, test_X) # merge X_train and X_test for the moment to compute features

In [ ]:
# Plot some of the pictures
layout(matrix(1:4, nrow = 2))

par(mar = c(2, 2, 2, 2))
as_image(X[41,])
mtext(train_y[41])

par(mar = c(2, 2, 2, 2))
as_image(X_test[41,])
mtext(test_y[41])

par(mar = c(2, 2, 2, 2))
as_image(X[499,])
mtext(train_y[499])

par(mar = c(2, 2, 2, 2))
as_image(X_test[499,])
mtext(test_y[499])

_Note: It looks like some of the picture labels are wrong. We could not figure out where we went wrong because we did not change the code we were given._

# 3. Feature Extraction

To classify the pictures we need to extract features from them. They need to be invariant to transformations like flipping, skewing, ... to accomodate the wide variety in pictures and faces. To do so will not only use the "raw" pixels we extracted above, but also so-called "edges". Edges represent areas where the pixel values change abruptly. This gives an outline of the pictures based on significant changes in intensity, color, or texture of the image. 

## 3.1 Function for extracting summary statistics

To make the working process easier, we wrote a function that automatically extracts basic summary statistics like mean, variance, standard deviation, ... . Additionally, we defined some functions ourselves. 

### 3.1.1 Entropy

In [ ]:
entropy <- function(X, nbreaks = 30) {
    r <- range(X)
    x_binned <- findInterval(X, seq(r[1], r[2], length.out = nbreaks))
    h <- tabulate(x_binned, nbins = nbreaks) # fast histogram
    p <- h / sum(h)
    -sum(p[p > 0] * log(p[p > 0]))
}

### 3.1.2 Combining it all in one function

In [ ]:
stats <- function(input_df, suffix) {
    
  # Calculate mean, variance, standard deviation, ... per row
    meansX <- rowMeans(input_df)
    # meansY <- colMeans(input_df)
    variancesX <- apply(input_df, 1, var)
    # variancesY <- apply(input_df, 2, var)
    std_dev <- apply(input_df, 1, sd)
    mins <- apply(input_df, 1, min)
    maxs <- apply(input_df, 1, max)
    medians <- apply(input_df, 1, median)
    entropies <- apply(input_df, 1, entropy)
    

  # Create a new dataframe with modified column names
  result_df <- data.frame(
    MeanX = meansX,
    #  MeanY = meansY,
    VarianceX = variancesX,
    # VarianceY = variancesY,
    StandardDeviation = std_dev,
    Minima = mins,
    Maxima = maxs,
    Width = maxs - mins, 
    Medians = medians,
    Entropy = entropies
  )

  # Add the specified suffix to column names
  colnames(result_df) <- paste0(colnames(result_df), "_", suffix)

  return(result_df)
}


## 3.2 Raw Pixel Analysis

Before analyzing edges, we applied some basic summary statistics to the raw pixels 


In [ ]:
train_stats_features_pixels <- stats(train_X,"pixel")
test_stats_features_pixels <- stats(test_X,"pixel")
head(train_stats_features_pixels, 3)

## 3.3 Edge Analysis

### 3.3.1 Extracting Edges

In [ ]:
# Define the threshold
threshold <- 0.0625

# Function to perform edge detection
edge_detection <- function(image) {
    
  h_edge <- image[-1,] - image[-nrow(image),]
  v_edge <- image[,-1] - image[,-ncol(image)]
  d_edge <- h_edge[,-1] - h_edge[,-ncol(h_edge)]

  # Apply the threshold
  h_edgeT <- h_edge < threshold
  v_edgeT <- v_edge < threshold
  d_edgeT <- d_edge < threshold / 2

  # Return the results as a list
  return(list(horizontal = h_edge, vertical = v_edge, diagonal = d_edge, 
              horizontalThreshold = h_edgeT, verticalThreshold = v_edgeT, 
              diagonalThreshold = d_edgeT))
}

### 3.3.2 Raw edges

In [ ]:
# Flatten and convert list of matrices from edge_detection into a numeric vector
flatten_edges <- function(edge_list) {
  # Convert each matrix in the list into a vector and concatenate them all into a single vector
  return(c(as.vector(edge_list$horizontal), 
           as.vector(edge_list$vertical),
           as.vector(edge_list$diagonal)))
}

# Extract Edge Features
train_edge_features <- lapply(1:nrow(train_X), function(i) {
  im <- matrix(train_X[i,], 48) 
  edges <- edge_detection(im)
  return(flatten_edges(edges))
})
test_edge_features <- lapply(1:nrow(test_X), function(i) {
  im <- matrix(test_X[i,], 48) 
  edges <- edge_detection(im)
  return(flatten_edges(edges))
})


In [ ]:
# Function to visualize the original and edge images
visualize_edge_features <- function(original_matrix, edge_features, index) {
  # Extract original image and reshape it to a 48x48 matrix
  original_image <- matrix(original_matrix[index,], 48)
  
  # Extract and reshape edge features to their respective image shapes
  h_edge_image <- matrix(edge_features[[index]][1:(48*47)], 48, 47, byrow = TRUE)
  v_edge_image <- matrix(edge_features[[index]][(48*47 + 1):(48*47 + 48*47)], 48, 47, byrow = TRUE)
  d_edge_image <- matrix(edge_features[[index]][(2*48*47 + 1):(2*48*47 + 47*47)], 47, 47, byrow = TRUE)

  # Plotting
  p1 <- ggplot() +
    geom_raster(data = NULL, aes(x = x, y = y, fill = original), interpolate = TRUE) +
    scale_fill_gradient(low = "black", high = "white") +
    ggtitle("Original Image") +
    theme_void() + theme(legend.position = "none") +
    coord_fixed(ratio = 1)
  p1$data <- expand.grid(x = 1:48, y = 1:48)
  p1$data$original <- as.vector(t(original_image))

  p2 <- ggplot() +
    geom_raster(data = NULL, aes(x = x, y = y, fill = h_edge), interpolate = TRUE) +
    scale_fill_gradient(low = "black", high = "white") +
    ggtitle("Horizontal Edges") +
    theme_void() + theme(legend.position = "none") +
    coord_fixed(ratio = 1)
  p2$data <- expand.grid(x = 1:48, y = 1:47)
  p2$data$h_edge <- as.vector(t(h_edge_image))

  p3 <- ggplot() +
    geom_raster(data = NULL, aes(x = x, y = y, fill = v_edge), interpolate = TRUE) +
    scale_fill_gradient(low = "black", high = "white") +
    ggtitle("Vertical Edges") +
    theme_void() + theme(legend.position = "none") +
    coord_fixed(ratio = 1)
  p3$data <- expand.grid(x = 1:47, y = 1:48)
  p3$data$v_edge <- as.vector(t(v_edge_image))

  p4 <- ggplot() +
    geom_raster(data = NULL, aes(x = x, y = y, fill = d_edge), interpolate = TRUE) +
    scale_fill_gradient(low = "black", high = "white") +
    ggtitle("Diagonal Edges") +
    theme_void() + theme(legend.position = "none") +
    coord_fixed(ratio = 1)
  p4$data <- expand.grid(x = 1:47, y = 1:47)
  p4$data$d_edge <- as.vector(t(d_edge_image))

  # Arrange the plots in a grid
  grid.arrange(p1, p2, p3, p4, ncol = 2)
}

# Use the function to visualize the 1st image and its edge features
visualize_edge_features(train_X, train_edge_features, 1)

In [ ]:
# Convert the list of edge features into a matrix
train_edge_features_matrix <- do.call(rbind, train_edge_features)
test_edge_features_matrix <- do.call(rbind, test_edge_features)

# Extract Statistical Features of edges
train_stats_features_edges <- stats(train_edge_features_matrix, "edge")
test_stats_features_edges <- stats(test_edge_features_matrix, "edge")

head(train_stats_features_edges, 3)

### 3.3.3 Thresholded edges

In [ ]:
# Apply the threshold to the matrix
train_edge_thresholded_matrix <- ifelse(train_edge_features_matrix < threshold, 0, 1)
test_edge_thresholded_matrix <- ifelse(test_edge_features_matrix < threshold, 0, 1)

train_stats_thresholded_edges <- stats(train_edge_thresholded_matrix, "thresholded_edges")
test_stats_thresholded_edges <- stats(test_edge_thresholded_matrix, "thresholded_edges")

head(train_stats_thresholded_edges, 3)

# 4. Creating and Cleaning Final Dataframe

## 4.1 Combining everything in one dataframe

In [ ]:
# Combine raw pixels, edge features, and their statistical features
train_X_combined <- cbind(train_edge_features_matrix, train_edge_thresholded_matrix, 
                          train_stats_features_pixels, train_stats_features_edges, train_stats_thresholded_edges)

test_X_combined <- cbind(test_edge_features_matrix, test_edge_thresholded_matrix,
                         test_stats_features_pixels, test_stats_features_edges, test_stats_thresholded_edges)


# Normalize the combined features
train_X_normalized <- scale(train_X_combined)
test_X_normalized <- scale(test_X_combined)

# Now train_X_normalized and test_X_normalized can be used for model training and testing

head(train_X_normalized, 3)

## 4.1 Removing near Zero-variance pixels

Pixels were removed due to near zero variance. For recognizing emotions through facial expressions the pixels that differ between emotions are limited. Many pixels, the ones that represent the outer part of the face, do not change by expressing an emotions. Those pixels can be removed.

In [ ]:
variance_threshold <- 0.045
variances <- apply(X, 2, var)

# Filter the columns based on the variance threshold
filtered_X <- X[, variances >= variance_threshold]

# Calculate the number of removed columns
num_removed_columns <- ncol(X) - ncol(filtered_X)

# Display the number of removed columns and the head of the original matrix
cat("Number of removed columns:", num_removed_columns, "\n")

### 3.5.1 plotting removed pixels

In [ ]:
# 1. Create a binary matrix of the same size as your image.
pixel_retention_matrix <- matrix(1, nrow=48, ncol=48)

# 2. Set the removed pixels to 0.
removed_pixel_indices <- which(variances < variance_threshold)
pixel_retention_matrix[removed_pixel_indices] <- 0

# 3. Plot the binary matrix.
library(ggplot2)

pixel_df <- data.frame(
  x = rep(1:48, each=48),
  y = rep(1:48, times=48),
  retained = as.vector(pixel_retention_matrix)
)

ggplot(pixel_df, aes(x=x, y=y, fill=factor(retained))) +
  geom_tile() +
  scale_fill_manual(values = c("red", "blue"), 
                    name = "Pixel Status",
                    breaks = c(0, 1),
                    labels = c("Removed", "Retained")) +
  coord_fixed(ratio = 1) +
  theme_void() +
  labs(title = "Pixels Removed Based on Variance Threshold")


In [ ]:
# Identify common columns
common_columns <- intersect(colnames(X), colnames(filtered_X))

# Subset X to keep only common columns
train_X <- train_X[, common_columns]
test_X <- test_X[, common_columns]

ncol(train_X) == ncol(filtered_X)
ncol(test_X) == ncol(filtered_X)

In [ ]:
# adding non-zero pixels to dataframe
train_X_combined <- cbind(train_X_combined, train_X)
test_X_combined <- cbind(test_X_combined, test_X)

head(train_X_combined, 3)

In [ ]:
# freeing up some RAM
# List the names of the objects you want to keep
objects_to_keep <- c("train_X_combined", "test_X_combined", "train_y", "test_y")

# List all objects in the current environment
all_objects <- ls()

# Remove all objects except the ones in objects_to_keep
objects_to_remove <- setdiff(all_objects, objects_to_keep)
rm(list = objects_to_remove)

## 4.3 High Correlations

In [ ]:
# high_corr <- findCorrelation(cor(train_X_combined), .95)

# cat("Deletes the following:", ifelse(!is.null(high_corr), paste(names(train_X_combined)[high_corr], collapse = ", "), "Deletes nothing"))

# CleanedFinalDf <- train_X_combined %>%
#  select(-all_of(high_corr)) 

# head(CleanedFinalDf, 6)

## 4.4 Normalizing Data

For some models the data needs to be normalized which we do in this very last step. 

In [ ]:
# Normalize the combined features
train_X_normalized <- scale(train_X_combined)
test_X_normalized <- scale(test_X_combined)

head(train_X_normalized, 3)

# 5. Model Fitting


## 5.1 Classification tree

We consider classification trees and random forests. Random forests are probably the least susceptible to overtraining and is considered one of the best "off the shelf" machine learning algorithms in the sense that they require little expertise in application, and easily perform well without tuning.

We fit a classification tree, using the pixel based approach. 

In [ ]:
suppressMessages(require(caret))

## Use multiple cores whenever possible
doMC::registerDoMC(cores = 4) 

## Fit a CART using 5-fold cross-validation to tune the complexity parameter
set.seed(2023) # for repeatability (generally pick the seed randomly)
trCntrl = trainControl('cv', 5, allowParallel = TRUE)
tt <- Sys.time()
fit_tree = train(x=train_X_normalized, y=train_y, method='rpart', trControl = trCntrl, tuneGrid = data.frame(cp=.02))
fit_tree

(dur <- Sys.time() - tt)

In [ ]:
## Graphical visualization of the decision tree
options(repr.plot.width=14, repr.plot.height=8)
plot(fit_tree$final, compress=TRUE, uniform=TRUE, margin=0.05, branch=.75); 
text(fit_tree$final, cex=0.8, all=TRUE, use.n=TRUE)

## Textual visualization of the decision tree
fit_tree$finalModel

The cross validated accuracy estimate is around 56%.

In [ ]:
## Check performance on the normalized training set
pred_tree = predict(fit_tree, train_X_normalized, type='raw') 
confusion_tree <- confusionMatrix(pred_tree, factor(train_y))


# Overall accuracy
accuracy_tree <- confusion_tree$overall['Accuracy']

# Print them
print(confusion_tree$table)
print(confusion_tree$byClass[,1:2])
print(paste("Overall Accuracy: ", round(accuracy_tree, 3)))

## 5.2 Ridge (Multinomial Regression with Ridge)

In [ ]:
# Speed up tuning by using all 4 CPU cores
doMC::registerDoMC(cores = 4)

#Performing ridge regression
fit_ridge <- glmnet::cv.glmnet(as.matrix(train_X_normalized),
                       factor(train_y),
                       family = "multinomial",
                       nfolds = 5,
                       alpha = 0,
                       type.measure = "class",
                       standardize = TRUE)

In [ ]:
# Performance evaluation CV Ridge
pred_ridge = predict(fit_ridge, 
                     as.matrix(test_X_normalized), 
                     s = 'lambda.min', 
                     type = 'class') %>% 
as.factor()

# Assessing Accuracy in Confusion-Matrix
confusion_ridge <- caret::confusionMatrix(pred_ridge,factor(test_y))

# Overall accuracy
accuracy_ridge <- confusion_ridge$overall['Accuracy']

# Print them
print(confusion_ridge$table)
print(confusion_ridge$byClass[,1:2])
print(paste("Overall Accuracy: ", round(accuracy_ridge, 3)))

## 5.3 Lasso (Multinomial Regression with Lasso)

In [ ]:
# Time how long the model-fitting takes
# Speed up tuning by using all 4 CPU cores
doMC::registerDoMC(cores = 4)

#Performing ridge regression
fit_lasso <- glmnet::cv.glmnet(as.matrix(train_X_normalized), 
                       factor(train_y), 
                       family = "multinomial",
                       nfolds = 5,
                       alpha = 1, 
                       type.measure = "class",
                       standardize = TRUE)


options(repr.plot.width = 7, repr.plot.height = 7)

# Plot misclassicication rate over the log of Lambda 
plot(fit_lasso)

In [ ]:
# Performance evaluation CV Lasso
pred_lasso = predict(fit_lasso, 
                     as.matrix(test_X_normalized), 
                     s = 'lambda.min', 
                     type = 'class') %>% 
as.factor()

# Assessing Accuracy in Confusion-Matrix
confusion_lasso = caret::confusionMatrix(pred_lasso,factor(test_y))

# Overall accuracy
accuracy_lasso <- confusion_lasso$overall['Accuracy']

# Print them
print(confusion_lasso$table)
print(confusion_lasso$byClass[,1:2])
print(paste("Overall Accuracy: ", round(accuracy_lasso, 3)))

## 5.4 Linear Discriminant Analysis (LDA)

In [ ]:
#trcntr <- trainControl('cv', 
#                       number = 2, 
#                       p = 0.8)

#fit_lda <- train(train_y ~ ., 
#                data = cbind(train_X_normalized, train_y), 
#                method = "lda", 
#                trControl = trcntr)

In [ ]:
# predict the validation set
#pred_lda <- predict(fit_lda, 
#                   test_X_normalized, 
#                   type='raw') 

# Assessing Accuracy in Confusion-Matrix
#confusion_lda = caret::confusionMatrix(pred_lda,factor(test_y))

# Overall accuracy
#accuracy_lda <- confusion_lda$overall['Accuracy']

# Print them
#print(confusion_lda$table)
#print(confusion_lda$byClass[,1:2])
#print(paste("Overall Accuracy: ", round(accuracy_lda, 3)))

## 5.5 Support Vector Machine (SVM)

In [ ]:
# note. for fitting this model we use the NOT normalized data because data preprocessing is built in this function. 
fit_svm <- train(x = train_X_combined, 
                 y = train_y, 
                 method = "svmRadial", 
                 trControl = trCntrl, 
                 preProcess = c("center","scale"), 
                 tuneLength = 10)

# Calculate predictions
pred_svm <- predict(fit_svm, test_X_combined)

# Assessing Accuracy in Confusion-Matrix
confusion_svm = caret::confusionMatrix(pred_svm,factor(test_y))

# Overall accuracy
accuracy_svm <- confusion_svm$overall['Accuracy']

# Print them
print(confusion_svm$table)
print(confusion_svm$byClass[,1:2])
print(paste("Overall Accuracy: ", round(accuracy_svm, 3)))

In [ ]:
## e1071

# fit model
tune_svm <- e1071::svm(x = train_X_normalized,
                       y = factor(train_y),
                       kernel = 'radial',
                       cost = 1,
                       scale = TRUE)

# Calculate predictions
pred_e1071 <- predict(tune_svm, test_X_normalized)

# Compute confusion matrix
Con_e1071 <- confusionMatrix(pred_e1071, factor(test_y))


# Assessing Accuracy in Confusion-Matrix
confusion_e1017 = caret::confusionMatrix(pred_e1071,factor(test_y))

# Overall accuracy
accuracy_e1017 <- confusion_e1017$overall['Accuracy']

# Print them
print(confusion_e1017$table)
print(confusion_e1017$byClass[,1:2])
print(paste("Overall Accuracy: ", round(accuracy_e1017, 3)))

# 6. Model Comparison

The last step was choosing the best out of the five models we fitted. <br>

## 6.1 Bias-Variance Trade-off
A general thing to consider during model selection between models with varying levels of flexibility is the bias-variance trade-off. The more flexible a model is, the less biased it is, but the more variance it has and vice versa. More biased models make stronger assumptions about the underlying data distribution. Because of this, they might fit the (training) data less well than less biased models if the assumed underlying distribution is incorrect. Models with high variance, often have the opposite problem: they might fit the (training) data very well but predict future cases less well because of overfitting.

As mentioned above, the different modelling algorithms we used are associated with different levels of flexibility. In contrast to previous competitions, the goal of this competition is to achieve a good prediction accuracy, instead of the correct classification of unknown test data. Therefore we do not see from using also more flexible models. 

## 6.2 Prediction Accuracy

Prediction Accuracy refers to the proportion of correctly classified cases. Results showed that there is a "clear winnder", a "clear looser" and three medium models. The classification tree model performed a lot worse than all other models with an accuracy of 0.65. The support vector model performed the best with an accuracy of 0.95. 

In [ ]:
FitMeasures <- data.frame(Model = c("Classification Tree", "Ridge", "Lasso", "Support Vector", "Tuned Support Vector"), 
                          Accuracy = c(accuracy_tree, accuracy_ridge, accuracy_lasso, accuracy_svm, accuracy_e1017))

# Create the ggplot bar plot
ggplot(FitMeasures, aes(x = Accuracy, y = Model)) +
  geom_bar(stat = "identity", fill = ifelse(FitMeasures$Accuracy == max(FitMeasures$Accuracy), "blue", "gray")) +
  geom_text(aes(label = round(Accuracy, 3)), hjust = -0.1) +
  labs(title = "Test Set Accuracy", x = "Accuracy") +
  xlim(0, 1.1) +
  theme_minimal() +
  coord_flip()

## 6.4 Cohen's Kappa

Cohen's Kappa is a measure of reliability of a model. The higher Kappa, the better the model. The pattern of Kappa values of the models is the same as for the model accuracies: The classification tree model performed very bad; lasso, ridge and tuned support vector medium, and support vector very well. So again, the support vector model is to be preferred.

In [ ]:
FitMeasures <- FitMeasures %>%
        mutate(Kappa = c(confusion_tree$overall[["Kappa"]], confusion_ridge$overall[["Kappa"]], confusion_lasso$overall[["Kappa"]],
                         confusion_svm$overall[["Kappa"]], confusion_e1017$overall[["Kappa"]]))


# Create the ggplot bar plot
ggplot(FitMeasures, aes(x = Kappa, y = Model)) +
  geom_bar(stat = "identity", fill = ifelse(FitMeasures$Kappa == max(FitMeasures$Kappa), "blue", "gray")) +
  geom_text(aes(label = round(Kappa, 3)), hjust = -0.1) +
  labs(title = "Test Set Kappa", x = "Kappa") +
  xlim(0, 1.1) +
  theme_minimal() +
  coord_flip()


## 6.3 Sensitivity

Sensitvitiy refers to the proportion of true positive cases among all actual positives, indicating how well a model detects true cases and avoids false negatives. We observed the exact same pattern as with the other fit measures and again decided on the support vector model. 

In [ ]:
FitMeasures <- FitMeasures %>%
                    mutate(Sensitivity = c(mean(confusion_tree$byClass[,"Sensitivity"]), mean(confusion_ridge$byClass[, "Sensitivity"]), mean(confusion_lasso$byClass[,"Sensitivity"]), 
                                           mean(confusion_svm$byClass[, "Sensitivity"]), mean(confusion_e1017$byClass[, "Sensitivity"])))


# Create the ggplot bar plot
ggplot(FitMeasures, aes(x = Sensitivity, y = Model)) +
  geom_bar(stat = "identity", fill = ifelse(FitMeasures$Sensitivity == max(FitMeasures$Sensitivity), "blue", "gray")) +
  geom_text(aes(label = round(Sensitivity, 3)), hjust = -0.1) +
  labs(title = "Test Set Sensitivity", x = "Sensitivity") +
  xlim(0, 1.1) +
  theme_minimal() +
  coord_flip()

## 6.4 Specificity

Specificity refers to the proportion of true negative cases among all actual negatives, indicating how well a model avoids "false alarms". Interestingly, there are no huge differences in specificity between the models. The model that again had a slight advantage over all other models was the support vector model.

In [ ]:
FitMeasures <- FitMeasures %>%
                    mutate(Specificity = c(mean(confusion_tree$byClass[,"Specificity"]), mean(confusion_ridge$byClass[, "Specificity"]), mean(confusion_lasso$byClass[,"Specificity"]), 
                                           mean(confusion_svm$byClass[, "Specificity"]), mean(confusion_e1017$byClass[, "Specificity"])))


# Create the ggplot bar plot
ggplot(FitMeasures, aes(x = Specificity, y = Model)) +
  geom_bar(stat = "identity", fill = ifelse(FitMeasures$Specificity == max(FitMeasures$Specificity), "blue", "gray")) +
  geom_text(aes(label = round(Specificity, 3)), hjust = -0.1) +
  labs(title = "Test Set Specificity", x = "Specificity") +
  xlim(0, 1.1) +
  theme_minimal() +
  coord_flip()

## 6.5. Final Model Choice

Taking the bias-variance tradeoff, accuracy, sensitivity and specificity into account, we decided on the Support Vector Model as our final model. <br>

# 7. Submission

## 7.1 Refit Support Vector Model on all data

In [ ]:
# Read in the images as pixel values (discarding color channels)
X = sapply(train_image_files, function(nm) c(readPNG(nm)[,,1])) %>% t() 
y = c(rep("anger", length(anger)), rep("happy", length(happy)), rep("sad", length(sad)), rep("disgust", length(disgust)))

X_test = sapply(test_im, function(nm) c(readPNG(nm)[,,1])) %>% t() 


# Change row and column names of X to something more managable (caret::train requires column names)
rownames(X)      = gsub(".+x/", "", rownames(X))
rownames(X_test) = gsub(".+x/",  "", rownames(X_test))
                
colnames(X) = colnames(X_test) = paste("p",1:ncol(X), sep="")

# Extract Edge Features
final_train_edge_features <- lapply(1:nrow(X), function(i) {
  im <- matrix(X[i,], 48) 
  edges <- edge_detection(im)
  return(flatten_edges(edges))
})
final_test_edge_features <- lapply(1:nrow(X_test), function(i) {
  im <- matrix(X_test[i,], 48) 
  edges <- edge_detection(im)
  return(flatten_edges(edges))
})

                
# Extract Statistical Features
final_train_stats_features <- stats(X, "x")
final_test_stats_features <- stats(X_test, "x")
                
final_train_edge_features_matrix <- do.call(rbind, final_train_edge_features)
final_test_edge_features_matrix <- do.call(rbind, final_test_edge_features)

final_train_stats_features_edges <- stats(final_train_edge_features_matrix, "x")
final_test_stats_features_edges <- stats(final_test_edge_features_matrix, "x")

# Combine the datasets
final_train_X_combined <- cbind(X, final_train_edge_features_matrix, final_train_stats_features, final_train_stats_features_edges)
final_test_X_combined <- cbind(X_test, final_test_edge_features_matrix, final_test_stats_features, final_test_stats_features_edges)
             
# Speed up tuning by using all 4 CPU cores
doMC::registerDoMC(cores = 4)
                
# refit ... on all observations
final_fit_svm <- train(x = final_train_X_combined, 
                 y = y, 
                 method = "svmRadial", 
                 trControl = trCntrl, 
                 preProcess = c("center","scale"), 
                 tuneLength = 10)

The statistics of the new dataset (included the edges) are not computed succesfully.

In [ ]:
# Combine all features
final_train_X_with_edges <- cbind(X, train_edge_features)
final_test_X_with_edges <- cbind(X_test, test_edge_features)

# Extract Statistical Features
final_train_stats_features <- stats(final_train_X_with_edges, "x")
final_test_stats_features <- stats(final_test_X_with_edges, "x")

# Combine all features
final_train_X_combined <- cbind(X, do.call(rbind, train_edge_features), train_stats_features)
final_test_X_combined <- cbind(X_test, do.call(rbind, test_edge_features), test_stats_features)
                
                
# Extract Statistical Features
final_train_stats_features <- stats(X, "x")
final_test_stats_features <- stats(X_test, "x")

                
# Combine all features
final_train_X_combined <- cbind(X, do.call(rbind, final_train_edge_features), final_train_stats_features)
final_test_X_combined <- cbind(X_test, do.call(rbind, final_test_edge_features), final_test_stats_features)
                
                                
                # refit ... on all observations
final_fit_svm <- train(x = final_train_X_combined, 
                 y = y, 
                 method = "svmRadial", 
                 trControl = trCntrl, 
                 preProcess = c("center","scale"), 
                 tuneLength = 10)

## 7.2 Create Submission

In [ ]:
## Make predictions
pred_svm = predict(final_fit_svm, final_test_X_combined, type='raw')

## Write to file
tibble(file = rownames(X_test), category = pred_svm) %>% 
    write_csv(path = "submission.csv")

## Check result
cat(readLines("submission.csv",n=20), sep="\n")

# 8. Division of Labor

* Sophia: Notebook, Model Fitting
* Janne: Notebook, Model Fitting, Features
* Leonie: Notebook, Features, Model Comparison